In [1]:
from langchain_openai import ChatOpenAI
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent 
import os
import getpass

model = ChatOpenAI(model="gpt-4o")

os.environ["LANGSMITH_TRACING"] = "true"
if not os.environ.get("LANGSMITH_API_KEY"):
  os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter API key for Langsmith: ")
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [2]:
def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

def web_search(query: str) -> str:
    """Search the web for information."""
    return (
        "Here are the headcounts for each of the FAANG companies in 2024:\n"
        "1. **Facebook (Meta)**: 63,317 employees. \n"
        "2. **Apple**: 147,000 employees. \n"
        "3. **Amazon**: 1,298,000 employees. \n"
        "4. **Netflix**: 10,400 employees. \n"
        "5. **Google (Alphabet)**: 160,000 employees."
    )

In [3]:
math_agent = create_react_agent(
    model=model,
    tools=[add, multiply],
    name="math_expert",
    prompt="You are a math expert. Always use one tool at a time."
)

research_agent = create_react_agent(
    model=model,
    tools=[web_search],
    name="research_expert",
    prompt="You are a research expert with access to web search. Do not do any math."
)

workflow = create_supervisor(
    [math_agent, research_agent],
    model=model,
    prompt=(
        "You are a team supervisor managing a research expert and a math expert."
        "For current events, use research_agent. "
        "For math problems, use math_agent."
    )
)

In [4]:
app = workflow.compile()
app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "What's the combined headcount of the FAANG companies in 2024?"
        }
    ]
})

{'messages': [HumanMessage(content="What's the combined headcount of the FAANG companies in 2024?", additional_kwargs={}, response_metadata={}, id='bf7fbb39-e02b-4da3-bd68-264e7cb8aceb'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2uvS66TOxM2hMk1eQluQRmEe', 'function': {'arguments': '{}', 'name': 'transfer_to_research_expert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 102, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb9dce56a8', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run-42ad59b9-4d51-4609-a694-81541601dd8d-0', tool_calls=[{'name': 'transfer_to_research_expert', 'args': {}, 'id': 'call_2uvS66TOxM2hMk1eQluQRmEe'